https://www.kaggle.com/datasets/cdc/behavioral-risk-factor-surveillance-system?select=2015.csv

https://www.cdc.gov/brfss/annual_data/2015/pdf/codebook15_llcp.pdf

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Cargar dataset
df_original = pd.read_csv('raw_diabetes_data_2015.csv')

# Visualizar dataset
df_original


,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENUM,...,_PAREC1,_PASTAE1,_LMTACT1,_LMTWRK1,_LMTSCL1,_RFSEAT2,_RFSEAT3,_FLSHOT6,_PNEUMO2,_AIDTST3
0,1.0,1.0,b'01292015',b'01',b'29',b'2015',1200.0,2.015000e+09,2.015000e+09,1.0,...,4.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,1.0
1,1.0,1.0,b'01202015',b'01',b'20',b'2015',1100.0,2.015000e+09,2.015000e+09,1.0,...,2.0,2.0,3.0,3.0,4.0,2.0,2.0,NaN,NaN,2.0
2,1.0,1.0,b'02012015',b'02',b'01',b'2015',1200.0,2.015000e+09,2.015000e+09,1.0,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,NaN
3,1.0,1.0,b'01142015',b'01',b'14',b'2015',1100.0,2.015000e+09,2.015000e+09,1.0,...,4.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,9.0
4,1.0,1.0,b'01142015',b'01',b'14',b'2015',1100.0,2.015000e+09,2.015000e+09,1.0,...,4.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441451,72.0,11.0,b'12162015',b'12',b'16',b'2015',1100.0,2.015005e+09,2.015005e+09,NaN,...,4.0,2.0,2.0,2.0,3.0,1.0,1.0,2.0,2.0,2.0
441452,72.0,11.0,b'12142015',b'12',b'14',b'2015',1100.0,2.015005e+09,2.015005e+09,NaN,...,2.0,2.0,3.0,3.0,4.0,1.0,1.0,NaN,NaN,1.0
441453,72.0,11.0,b'12232015',b'12',b'23',b'2015',1200.0,2.015005e+09,2.015005e+09,NaN,...,9.0,9.0,3.0,3.0,4.0,9.0,9.0,9.0,9.0,NaN
441454,72.0,11.0,b'12152015',b'12',b'15',b'2015',1100.0,2.015005e+09,2.015005e+09,NaN,...,4.0,2.0,3.0,3.0,4.0,1.0,1.0,NaN,NaN,2.0


In [2]:
def reemplazar_valores_binarios(valor):
    try:
        valor = float(valor)
        if valor == 1:
            return 1
        elif valor == 2:
            return 0
        elif valor in [7, 9]:
            return np.nan
        else:
            return np.nan
    except:
        return np.nan

# Convierte códigos específicos en frecuencia diaria aproximada
def convertir_a_veces_dia(valor):
    if pd.isna(valor):
        return np.nan
    valor = int(valor)
    if valor == 555:
        return 0  # Nunca o no aplica
    if valor in [777, 999, 300]:
        return 0.01  # Casi nunca
    if 101 <= valor <= 199:
        return valor - 100  # Frecuencia diaria directa
    elif 201 <= valor <= 299:
        return (valor - 200) / 7  # Frecuencia semanal convertida a diaria
    elif 301 <= valor <= 399:
        return (valor - 300) / 30  # Frecuencia mensual convertida a diaria
    else:
        return np.nan  # Código no reconocido

# Clasifica la frecuencia diaria en categorías de consumo
def convertir_a_rango(veces_dia):
    if pd.isna(veces_dia):
        return np.nan
    veces_semana = veces_dia * 7
    if veces_semana >= 7:
        return 1  # Consumo alto (diario)
    elif veces_semana >= 3:
        return 2  # Consumo medio (cada 2–3 días)
    elif veces_semana >= 1:
        return 3  # Consumo bajo (una vez por semana)
    else:
        return 4  # Muy bajo o nunca

# Lista de columnas originales relevantes para análisis
columnas_seleccionadas = [
    'DIABETE3',     # Estado de diabetes
    '_RFHYPE5',     # Presión arterial alta
    'TOLDHI2',      # Colesterol alto
    'BLOODCHO',     # Control de colesterol realizado
    '_BMI5',        # Índice de masa corporal (multiplicado por 100)
    'SMOKE100',     # Ha fumado al menos 100 cigarrillos en la vida
    'CVDSTRK3',     # Historial de accidente cerebrovascular
    'CVDCRHD4',     # Historial de enfermedad cardíaca o infarto
    'EXERANY2',     # Realiza actividad física
    'FRUIT1',       # Consumo diario de frutas
    'VEGETAB1',     # Consumo diario de verduras
    'GENHLTH',      # Percepción de salud general
    'PHYSHLTH',     # Días con mala salud física
    'MENTHLTH',     # Días con mala salud mental
    'DIFFWALK',     # Dificultad para caminar
    'SEX',          # Sexo
    '_AGEG5YR',     # Grupo de edad
    'EDUCA',        # Nivel educativo
    'INCOME2',      # Categoría de ingresos
    '_RACEGR3'      # Grupo racial
]

# Crear un nuevo DataFrame con solo las columnas seleccionadas
df_seleccionado = df_original[columnas_seleccionadas].copy()

# Mapeo para renombrar columnas a formato más legible (snake_case)
mapa_renombrado = {
    'DIABETE3': 'estado_diabetes',
    '_RFHYPE5': 'presion_alta',
    'TOLDHI2': 'colesterol_alto',
    'BLOODCHO': 'control_colesterol',
    '_BMI5': 'bmi',
    'SMOKE100': 'fumo_100_cigs',
    'CVDSTRK3': 'historial_acv',
    'CVDCRHD4': 'historial_cardiaco',
    'EXERANY2': 'actividad_fisica',
    'FRUIT1': 'frutas_diarias',
    'VEGETAB1': 'verduras_diarias',
    'GENHLTH': 'salud_general',
    'PHYSHLTH': 'dias_mala_salud_fisica',
    'MENTHLTH': 'dias_mala_salud_mental',
    'DIFFWALK': 'dificultad_caminar',
    'SEX': 'sexo',
    '_AGEG5YR': 'grupo_edad',
    'EDUCA': 'nivel_educativo',
    'INCOME2': 'categoria_ingresos',
    '_RACEGR3': 'grupo_racial'
}

df_seleccionado.rename(columns=mapa_renombrado, inplace=True)

# Crear columna binaria para diabetes: 0 = no diabetes, 1 = prediabetes o diabetes
# En el dataset original: 1 = no diabetes, 2 = prediabetes, 3 = diabetes
df_seleccionado['diabetes_bin'] = df_seleccionado['estado_diabetes'].apply(lambda x: 1 if x in [2, 3] else 0)

# Eliminar la columna original no binaria de diabetes
df_seleccionado.drop(columns=['estado_diabetes'], inplace=True)

# Ajustar índice de masa corporal dividiendo por 100 para obtener valor real
df_seleccionado['bmi'] = df_seleccionado['bmi'] / 100

# Reemplazo de valores en presión arterial alta (invertir códigos para facilitar análisis)
df_seleccionado['presion_alta'] = df_seleccionado['presion_alta'].replace({1: 0, 2: 1, 9: np.nan})

# Aplicar función de reemplazo binario a colesterol alto y control de colesterol
df_seleccionado['colesterol_alto'] = df_seleccionado['colesterol_alto'].apply(reemplazar_valores_binarios)
df_seleccionado['control_colesterol'] = df_seleccionado['control_colesterol'].apply(reemplazar_valores_binarios)

# Convertir consumo diario de frutas y verduras a categorías de frecuencia
df_seleccionado['frecuencia_frutas'] = df_seleccionado['frutas_diarias'].apply(convertir_a_veces_dia).apply(convertir_a_rango)
df_seleccionado['frecuencia_verduras'] = df_seleccionado['verduras_diarias'].apply(convertir_a_veces_dia).apply(convertir_a_rango)

# Eliminar las columnas originales ya procesadas
df_seleccionado.drop(columns=['frutas_diarias', 'verduras_diarias'], inplace=True)

# Mostrar conteo de valores faltantes por columna
print("Valores faltantes por columna:")
print(df_seleccionado.isnull().sum())

# Opcional: eliminar filas con valores faltantes
df_seleccionado.dropna(inplace=True)

# Mostrar tamaño y primeras filas del DataFrame limpio
print(f"Dimensiones finales del dataframe: {df_seleccionado.shape}")
print(df_seleccionado.head())

# Verificar y eliminar filas duplicadas
num_duplicados = df_seleccionado.duplicated().sum()
print(f"Número de filas duplicadas: {num_duplicados}")

df_sin_duplicados = df_seleccionado.drop_duplicates()
df_sin_duplicados.reset_index(drop=True, inplace=True)

# Guardar DataFrame limpio a CSV (opcional)
df_sin_duplicados.to_csv('cleaned_diabetes_data_2015.csv', index=False)


Valores faltantes por columna:
presion_alta               1367
colesterol_alto           62715
control_colesterol         9481
bmi                       36398
fumo_100_cigs             14255
historial_acv                 0
historial_cardiaco            1
actividad_fisica          35444
salud_general                 2
dias_mala_salud_fisica        1
dias_mala_salud_mental        0
dificultad_caminar        12334
sexo                          0
grupo_edad                    0
nivel_educativo               0
categoria_ingresos         3301
grupo_racial                  0
diabetes_bin                  0
frecuencia_frutas         29150
frecuencia_verduras       33138
dtype: int64
Dimensiones finales del dataframe: (328053, 20)
   presion_alta  colesterol_alto  control_colesterol    bmi  fumo_100_cigs  \
0           1.0              1.0                 1.0  40.18            1.0   
1           0.0              0.0                 1.0  25.09            1.0   
3           1.0              1.0  